In [1]:
import asyncio

In [2]:
"""
先用最简单的方式实现：
    thread的启停（使用state?保存函数名称，在外部。
    timer的加入
    button sync
    list button
"""
None

In [4]:
class State:
    funclist=[]    #for TaskHandler


In [5]:
class TaskHandler:
    def __init__(self,func,*args,**kwargs):
        self.func=func
        self.task=None
        self.args=args
        self.kwargs=kwargs
    def start(self,l=None):
        if not self.is_running():
            self.task=asyncio.create_task(self.func(*self.args,**self.kwargs))
    
    error=None
    def is_running(self):
        if self.task is None:return False
        if not self.task.done():return True
        self.end()
        del self.task
        self.task=None
        
        
    
    def cancel(self):
        if not self.is_running():return
        self.task.cancel()
        
            
    def stop(self,l=None):
        self.cancel()
        
    def end(self):
        try:self.task.result()
        except asyncio.CancelledError:pass
        except asyncio.InvalidStateError:print("result not good")
        except Exception as e:self.error=e

    name=""
    def __str__(self):
        return self.name

In [6]:
from functools import wraps

def add_to_state(state):
    def wrapper(func):
        @wraps(func)
        async def wrapped(*args, **kwargs):
            state.funclist.append(func(*args, **kwargs))
            return await func(*args, **kwargs)
        return wrapped
    return wrapper
def add_to_state(state):
    def wrapper(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            state.funclist.append(TaskHandler(func,*args, **kwargs))
            return
        return wrapped
    return wrapper

In [7]:
class StateT(State):
    def find_func_by_name():
#         for i in 
        ...
    def start_func():
        ...


s=StateT()

In [8]:
import ipywidgets
class TaskHandlerButton(TaskHandler):
    def __init__(self,func,*args,**kwargs):
        self.func=func
        self.task=None
        self.args=args
        self.kwargs=kwargs
        self.create_buttons()
        
    def create_buttons(self):
        self.start_button = ipywidgets.Button(description="start")
        self.start_button.on_click(self.start)
        self.stop_button = ipywidgets.Button(description="stop")
        self.stop_button.on_click(self.stop)
        self.box=ipywidgets.Box([self.start_button,self.stop_button])
        
    def display(self):
        display(self.box)

def add_to_state_button(state):
    def wrapper(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            state.funclist.append(TaskHandlerButton(func,*args, **kwargs))
            return
        return wrapped
    return wrapper

In [9]:
async def sync_value(from_value,to_value,
                     from_value_name="value",to_value_name="value",
                     sync_time=0.01):
    while 1:
        await asyncio.sleep(sync_time)
        if hasattr(to_value,to_value_name) and hasattr(from_value,from_value_name):break
    while 1:
        await asyncio.sleep(sync_time)
        x=getattr(from_value,from_value_name)
        if x==getattr(to_value,to_value_name):continue
        setattr(to_value,to_value_name,x)



In [10]:
@add_to_state_button(s)
async def loop_timer(state,sleep_interval=0.1,keep_len=10):
    #TODO: better timing function to reduce execution time
    from collections import deque
    import time
    timed_list=deque((),keep_len)

    state.loop_time=-1
    
    for _ in range(keep_len):
        await asyncio.sleep(sleep_interval)
        timed_list.append(time.time())
        
    while 1:
        await asyncio.sleep(sleep_interval)
        t=time.time()
        state.loop_time=t-timed_list.popleft()
        timed_list.append(t)
loop_timer(s)

In [11]:
t=ipywidgets.BoundedFloatText()
thing1=asyncio.create_task(sync_value(s,t,"loop_time"))
t

BoundedFloatText(value=0.0)

In [12]:
t.value=1

In [13]:
s.funclist

In [14]:
go=s.funclist[0]

In [15]:
go.display()

Box(children=(Button(description='start', style=ButtonStyle()), Button(description='stop', style=ButtonStyle()…

In [16]:
go.is_running()

False

In [17]:
go.task

In [18]:
go.task.result

AttributeError: 'NoneType' object has no attribute 'result'

In [ ]:
go.func

In [ ]:
break

In [ ]:
asyncio.create_task(go.func)

In [ ]:
go.task=asyncio.create_task(go.func)

In [ ]:
go.task

In [ ]:
go.task.done()

In [ ]:
go.error

In [ ]:
dir(ipywidgets)

In [ ]:
def cancel_from_funclist():
    for i in asyncio.all_tasks():
        